In [13]:
# Setup the Jupyter version of Dash
from jupyter_dash import JupyterDash

# Configure the necessary Python module imports for dashboard components
import base64
import dash
from dash import dash_table, dcc, html
from dash.dependencies import Input, Output, State
import dash_leaflet as dl
import plotly.express as px
JupyterDash.infer_jupyter_proxy_config()

# Configure OS routines
import os

# Configure the plotting routines
import numpy as np
import pandas as pd

# CRUD Python module
from CRUD_Python_Module import AnimalShelter

###########################
# Data Manipulation / Model
###########################
username = 'aacuser'
password = 'testPassword123'

# Connect to database via CRUD Module
db = AnimalShelter(username, password, 'localhost', 27017, 'aac', 'animals')

# class read method must support return of list object and accept projection json input
# sending the read method an empty document requests all documents be returned
df = pd.DataFrame.from_records(db.read({}))

# MongoDB v5+ is going to return the '_id' column and that is going to have an 
# invlaid object type of 'ObjectID' - which will cause the data_table to crash - so we remove
# it in the dataframe here. The df.drop command allows us to drop the column. If we do not set
# inplace=True - it will reeturn a new dataframe that does not contain the dropped column(s)
df.drop(columns=['_id'],inplace=True)

## Debug
# print(len(df.to_dict(orient='records')))
# print(df.columns)


#########################
# Dashboard Layout / View
#########################
app = JupyterDash(__name__)

# Grazioso Salvare’s logo
image_filename = 'Grazioso Salvare Logo.png'
encoded_image = base64.b64encode(open(image_filename, 'rb').read())
brandbar = html.Div(
    style={'display': 'flex', 'alignItems': 'center', 'gap': '16px', 'justifyContent': 'space-between'},
    children=[
        html.A(
            href='https://www.snhu.edu',
            target='_blank',
            children=html.Img(
                src='data:image/png;base64,{}'.format(encoded_image.decode()),
                alt='Grazioso Salvare logo',
                style={'height': '64px'}
            )
        ),
        html.Div(
            style={'fontWeight': '600'},
            children='reeduxx @ GitHub'
        )
    ]
)

app.layout = html.Div([
#    html.Div(id='hidden-div', style={'display':'none'}),
    brandbar,
    html.Center(html.B(html.H1('CS-340 Dashboard'))),
    html.Hr(),
    html.Div(
        id='filters',
        style={'display': 'flex', 'gap': '24px', 'flexWrap': 'wrap', 'alignItems': 'center', 'justifyContent': 'center'},
        children=[
            dcc.RadioItems(
                id='filter-type',
                options=[
                    {'label': 'Water Rescue', 'value': 'water'},
                    {'label': 'Mountain/Wilderness Rescue', 'value': 'mountain'},
                    {'label': 'Disaster Rescue/Individual Tracking', 'value': 'disaster'}
                ],
                value=None,
                labelStyle={'marginRight': '16px'}
            ),
            html.Button('Reset', id='reset-btn', n_clicks=0)
        ]
    ),
    html.Hr(),
    dash_table.DataTable(id='datatable-id',
                         columns=[{'name': i, 'id': i, 'deletable': False, 'selectable': True} for i in df.columns],
                         data=df.to_dict('records'),
                         page_size=10,
                         filter_action='native',
                         sort_action='native',
                         sort_mode='multi',
                         column_selectable='single',
                         row_selectable='single',
                         selected_rows=[0],
                         style_table={'height': '400px', 'overflowY': 'auto'},
                         style_cell={
                             'minWidth': '120px', 'width': '120px', 'maxWidth': '260px',
                             'whiteSpace': 'normal', 'textAlign': 'left'
                         },
                         style_header={'fontWeight': 'bold'},
                         virtualization=False,
                        ),
    html.Br(),
    html.Hr(),
#This sets up the dashboard so that your chart and your geolocation chart are side-by-side
    html.Div(className='row',
         style={'display' : 'flex'},
             children=[
        html.Div(
            id='graph-id',
            className='col s12 m6',

            ),
        html.Div(
            id='map-id',
            className='col s12 m6',
            )
        ])
])

#############################################
# Interaction Between Components / Controller
#############################################
@app.callback([Output('datatable-id', 'data'),
              Output('datatable-id', 'selected_rows'),
              Output('filter-type', 'value')],
              [Input('filter-type', 'value'),
              Input('reset-btn', 'n_clicks')])
def update_dashboard(filter_type, reset_clicks):
    triggered = dash.callback_context.triggered[0]['prop_id'].split('.')[0] if dash.callback_context.triggered else None
    
    if triggered == 'reset-btn':
               return df.to_dict('records'), ([0] if not df.empty else []), None
    
    base_query = {'animal_type': 'Dog'}
    water_breeds = [
        'Labrador Retriever Mix',
        'Chesapeake Bay Retriever',
        'Newfoundland'
    ]
    mountain_breeds = [
        'German Shepherd',
        'Alaskan Malamute',
        'Old English Sheepdog',
        'Siberian Husky',
        'Rottweiler'
    ]
    disaster_breeds = [
        'Doberman Pinscher',
        'German Shepherd',
        'Golden Retriever',
        'Bloodhound',
        'Rottweiler'
    ]
    
    if filter_type is None:
               return df.to_dict('records'), ([0] if not df.empty else []), None
    if filter_type == 'water':
        q = {
            **base_query,
            'breed': {'$in': water_breeds},
            'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156},
            'sex_upon_outcome': {'$regex': '^Intact Female'}
        }
    elif filter_type == 'mountain':
        q = {
            **base_query,
            'breed': {'$in': mountain_breeds},
            'age_upon_outcome_in_weeks': {'$gte': 26, '$lte': 156},
            'sex_upon_outcome': {'$regex': '^Intact Male'}
        }
    elif filter_type == 'disaster':
        q = {
            **base_query,
            'breed': {'$in': disaster_breeds},
            'age_upon_outcome_in_weeks': {'$gte': 20, '$lte': 300},
            'sex_upon_outcome': {'$regex': '^Intact Male'}
        }
    else:
        return df.to_dict('records'), ([0] if not df.empty else []), None
    
    results = db.read(q)
    dff = pd.DataFrame.from_records(results)
    
    if '_id' in dff.columns:
        dff.drop(columns=['_id'], inplace=True, errors='ignore')
    
    data = dff.to_dict('records') if not dff.empty else []
    selected = [0] if len(data) > 0 else []
    return data, selected, filter_type

# Display the breeds of animal based on quantity represented in
# the data table
@app.callback(
    Output('graph-id', "children"),
    [Input('datatable-id', "derived_virtual_data")])
def update_graphs(viewData):
    use_df = pd.DataFrame(viewData) if viewData is not None else df.copy()
    
    if use_df.empty or 'breed' not in use_df.columns:
        return [html.Div("No data to chart.")]
    
    counts = use_df['breed'].value_counts(dropna=False)
    topN = 10
    top = counts.head(topN)
    other_total = counts.iloc[topN:].sum()
    plot_df = top.reset_index()
    plot_df.columns = ['breed', 'count']
    
    if other_total > 0:
        plot_df = pd.concat(
            [plot_df, pd.DataFrame([{'breed': 'Other', 'count': int(other_total)}])],
            ignore_index=True
        )
    
    fig = px.pie(plot_df, names='breed', values='count', title='Selected Dogs by Breed (Top 10)')
    fig.update_traces(textposition='inside', textinfo='percent+label')
    fig.update_layout(legend=dict(orientation='h', yanchor='bottom', y=-0.15))
    return [dcc.Graph(figure=fig)]
    
#This callback will highlight a cell on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
)
def update_styles(selected_columns):
    selected_columns = selected_columns or []
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable
@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])
def update_map(viewData, index):  
    if viewData is None:
        return
    elif index is None:
        return 
    
    dff = pd.DataFrame.from_dict(viewData)
    # Because we only allow single row selection, the list can be converted to a row index here
    if index is None:
        row = 0
    else: 
        row = index[0]
        
    # Austin TX is at [30.75,-97.48]
    return [
        dl.Map(style={'width': '1000px', 'height': '500px'}, center=[30.75,-97.48], zoom=10, children=[
            dl.TileLayer(id="base-layer-id"),
            # Marker with tool tip and popup
            # Column 13 and 14 define the grid-coordinates for the map
            # Column 4 defines the breed for the animal
            # Column 9 defines the name of the animal
            dl.Marker(position=[dff.iloc[row,13],dff.iloc[row,14]], children=[
                dl.Tooltip(dff.iloc[row,4]),
                dl.Popup([
                    html.H1("Animal Name"),
                    html.P(dff.iloc[row,9])
                ])
            ])
        ])
    ]


# Run app and display result in jupyterlab mode, note, if you have previously run a prior app, the default port of 8050 may not be available, if so, try setting an alternate port.
app.run_server() 

Dash app running on https://wonderslalom-rubymodern-3000.codio.io/proxy/8050/
